# File with  base functions/types for my simulation

This includes:
* Types for Beliefs and Agents;
* Functions to create agents;
* Functions to create the network;
* Functions to update the agents' beliefs;
* Functions to store the simulation information

In [1]:
using LightGraphs, MetaGraphs, GraphPlot, Distributions, DataFrames

## Type declarations

The code primary elements are the the Belief and Agent types. A belief is a pair $(\sigma, \mu)$ of uncertainty and expected value of a distribution about a issue. So the type Belief has 3 fields. An Agent will have an id, a set of beliefs, an ideal point (which will be the mean of its opinions $\mu = o $ ) and a set of neighbors. The opinion will be generated from an Uniform(0,1) distribution, while the uncertainty will be , _in this version_, a global var.

In [2]:
#Structs for Agents and Beliefs --------------------

abstract type  AbstractAgent end
abstract type AbstractBelief end 


mutable struct Belief{T1 <: Real, T2 <: Integer}
    o::T1
    σ::T1
    whichissue::T2
end

mutable struct Agent{T1 <: Integer, T2 <: Vector, T3 <: Real, T4 <: Vector} <: AbstractAgent
    id::T1
    ideo::T2
    idealpoint::T3
    neighbors::T4
end


## Constructors for Beliefs, Agents and Graphs

In [3]:
function create_belief(σ::Real, issue::Integer)
    o = rand(Uniform())
    belief = Belief(o, σ, issue)
end

function create_idealpoint(ideology)
    opinions = []
    for issue in ideology
        push!(opinions,issue.o)
    end
    ideal_point = mean(opinions)
end

function create_agent(n_issues::Integer, id::Integer, σ::Real)
    ideology = [create_belief(σ, issue) for issue in 1:n_issues ]
    idealpoint = create_idealpoint(ideology)
    agent = Agent(id,ideology, idealpoint,[0])
end

function listofagents(σ::Real,  n_issues::Integer, size::Integer)
    population = [create_agent(n_issues,i,σ) for i in 1:size]
end


function add_neighbors!(pop, nw)
    for i in pop
        i.neighbors = neighbors(nw,i.id)
    end
end

function creategraphfrompop(pop)
    graphsize = length(pop)
    nw = CompleteGraph(graphsize)
    return(nw)
end


function setmgproperties!(mg::MetaGraph, pop)
    for i in pop
        set_props!(mg, i.id, Dict(:id => i.id, 
                :ideology => i.ideo,
                :idealpoint => i.idealpoint, 
                :neighbors => i.neighbors))
    end
end

setmgproperties! (generic function with 1 method)

## Interaction functions

In [4]:
function getjtointeract(i::Agent, metagraph)
    j = rand(props(metagraph, i.id)[:neighbors])
end

#Input = two agents; Output = a issue and associated beliefs
function pick_issuebelief(i::Agent, j::Agent, n_issues::Integer)
    issue_belief = rand(1:n_issues)
    i_belief = i.ideo[issue_belief]
    j_belief = j.ideo[issue_belief]
    return(issue_belief, i_belief, j_belief)
end

pick_issuebelief (generic function with 1 method)

In [7]:
population = listofagents(0.1, 2, 10)
g = creategraphfrompop(population)
add_neighbors!(population,g)
mg = MetaGraph(g)
setmgproperties!(mg,population)




In [8]:
props(mg, 1)

Dict{Symbol,Any} with 4 entries:
  :ideology   => Belief{Float64,Int64}[Belief{Float64,Int64}(0.342196, 0.1, 1), Belief{Float64,Int64}(0.210104, 0.1, 2)]
  :id         => 1
  :neighbors  => [2, 3, 4, 5, 6, 7, 8, 9, 10]
  :idealpoint => 0.27615